In [1]:
import pandas as pd
import requests
import time
from pathlib import Path

In [ ]:
# === Config ===
MODEL = "model-1"
EVAL_FILE = "../../data/processed/bdm-corpus-2/stage-3/test.csv"
RESULT_FILE = "../../results/exp002/results.csv"
TEMPERATURE = 0.0

In [3]:
eval_df = pd.read_csv(EVAL_FILE)

In [4]:
len(eval_df)

400

In [5]:
def build_prompt(headline: str) -> str:
    return (
        f"{headline.strip()}\n\n"
        f"Preveja o impacto direcional na taxa de câmbio USD/BRL com uma única palavra: Aumento ou Diminuição."
    )

In [6]:
# === Step 3: Query Ollama ===
def query_ollama(model, prompt):
    res = requests.post("http://localhost:11434/api/chat", json={
        "model": model,
        "temperature": TEMPERATURE,
        "stream": False,
        "messages": [{"role": "user", "content": prompt}]
    })
    res.raise_for_status()
    return res.json()["message"]["content"].strip()

In [7]:
# === Step 4: Run prediction loop ===
predictions = []
for i, row in eval_df.iterrows():
    prompt = build_prompt(row["Manchete"])
    try:
        out = query_ollama(MODEL, prompt)
    except Exception as e:
        out = "Erro"
    predictions.append(out)
    time.sleep(0.25)  # reduce overload risk

In [8]:
eval_df["Previsão"] = predictions
eval_df.to_csv(RESULT_FILE, index=False)
print(f"Saved predictions to {RESULT_FILE}")

Saved predictions to ../../results/exp002/results.csv


In [9]:
from sklearn.metrics import classification_report

# Reload the results file
df = pd.read_csv("../../results/exp002/results.csv")

# Normalize predictions (just in case)
df["Previsão"] = df["Previsão"].str.strip().str.capitalize()
df["Direção"] = df["Direção"].str.strip().str.capitalize()

# Report
report = classification_report(
    df["Direção"], df["Previsão"],
    labels=["Aumento", "Diminuição"],
    target_names=["Aumento", "Diminuição"],
    digits=3
)

print("\nClassification Report:\n")
print(report)


Classification Report:

              precision    recall  f1-score   support

     Aumento      0.539     0.205     0.297       200
  Diminuição      0.511     0.825     0.631       200

   micro avg      0.516     0.515     0.516       400
   macro avg      0.525     0.515     0.464       400
weighted avg      0.525     0.515     0.464       400



In [10]:
from sklearn.metrics import confusion_matrix
'''
[[TP FN]
 [FP TN]]
'''
cm = confusion_matrix(df["Direção"], df["Previsão"], labels=["Aumento", "Diminuição"])
print(cm)

[[ 41 158]
 [ 35 165]]
